In [1]:
#Bibliotecas 
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import SplineTransformer
from sklearn.model_selection import TimeSeriesSplit

In [2]:
#Dicionário de dados
df=pd.read_csv('../data/processed/data.csv')
df_dict = pd.read_csv('../data/external/dictionary.csv')
display(HTML(df_dict.to_html(index=False)))


variavel,descricao,tipo,subtipo
ID,Identificador único do sujeito,Quantitativo,Discreta
Age,Idade do sujeito,Quantitativo,Discreta
Gender,Gênero do sujeito,Qualitativo,Nominal
Bedtime,Hora em que o sujeito vai para a cama,Cíclica,Cíclica
Wakeup time,Hora em que o sujeito acorda,Cíclica,Cíclica
Sleep duration,Duração total do sono em horas,Cíclica,Cíclica
Sleep efficiency,Representa a porcentagem de tempo que o sujeito passou dormindo enquanto estava na cama,Quantitativo,Contínua
REM sleep percentage,"Porcentagem do tempo total de sono que foi gasto no estágio REM, fase do sono associada aos sonhos",Quantitativo,Contínua
Deep sleep percentage,"Porcentagem do tempo total de sono que foi gasto no estágio de sono profundo, fase do sono que é mais restauradora",Quantitativo,Contínua
Light sleep percentage,"Porcentagem do tempo total de sono que foi gasto no estágio de sono leve, fase do sono mais superficial",Quantitativo,Contínua


# 1. Variável-alvo e Variáveis preditoras

In [3]:
#Variável-alvo 
target_variable = "Sleep efficiency"

#Variável Nominais
nominal_variables=(
    df_dict
    .query("subtipo == 'Nominal' and variavel != @target_variable ")
    .variavel
    .to_list()
)

#Variável Cíclica
cyclic_variables=(
    df_dict
    .query("subtipo == 'Cíclica' and variavel != @target_variable ")
    .variavel
    .to_list()
)

#Variável Contínuas
continuous_variables=(
    df_dict
    .query("subtipo == 'Contínua' and variavel != @target_variable ")
    .variavel
    .to_list()
)

#Variável discreta
discrete_variables=(
    df_dict
    .query("subtipo == 'Discreta' and variavel != @target_variable ")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_variable], axis=1)
y = df[target_variable]


# 2. Pipeline de Pré-processamento de dados 

In [4]:
#Tratamento de dados faltantes e discrepantes, codificação e normalização

#Pipeline para variáveis nominais 
nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')),
    ("encoding", OneHotEncoder(sparse_output=False)),
])

#Pipeline para variáveis contínuas
continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')),
    ("normalization", StandardScaler()),
])

#Pipeline para variáveis discreta
discrete_preprocessor= Pipeline(steps=[
    ("missing", KNNImputer()),
    ("normalization", StandardScaler()),
])

#Pipeline para variáveis cíclicas
cyclic_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="mean")),  
    ("cyclical_encoding", TimeSeriesSplit()), 
])


In [5]:
preprocessor= ColumnTransformer([
    ("nominal", nominal_preprocessor, nominal_variables),
    ("continuous", continuous_preprocessor, continuous_variables),
    ("discrete", discrete_preprocessor, discrete_variables),
    ("cyclic", cyclic_preprocessor,cyclic_variables ),
])